In [1]:
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined"])
df['daily_pct_change'] = df['price'].pct_change() * 100
df = df[['date', 'price', 'daily_pct_change']]
df

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,date,price,daily_pct_change
0,2010-09-25,0.100000,NaN
1,2010-09-26,0.100000,0.000000
2,2010-09-27,0.100000,0.000000
3,2010-09-28,0.100000,0.000000
4,2010-09-29,0.100000,0.000000
...,...,...,...
5169,2024-12-17,106140.601562,0.104577
5170,2024-12-18,100041.539062,-5.746211
5171,2024-12-19,97490.953125,-2.549527
5172,2024-12-20,97755.929688,0.271796


In [16]:
# Define the date ranges for each cycle
cycle_1_start = '2011-11-26'
cycle_1_end = '2014-01-26'
cycle_2_start = '2015-01-14'
cycle_2_end = '2018-01-06'
cycle_3_start = '2019-02-07'
cycle_3_end = '2021-11-08'
cycle_4_start = '2022-11-22'
cycle_4_end = df['date'].max()

# Create data frames for each cycle
cycle_1 = df[(df['date'] >= cycle_1_start) & (df['date'] <= cycle_1_end)]
cycle_2 = df[(df['date'] >= cycle_2_start) & (df['date'] <= cycle_2_end)]
cycle_3 = df[(df['date'] >= cycle_3_start) & (df['date'] <= cycle_3_end)]
cycle_4 = df[(df['date'] >= cycle_4_start) & (df['date'] <= cycle_4_end)]
cycle_1.reset_index(drop=True, inplace=True)
cycle_2.reset_index(drop=True, inplace=True)
cycle_3.reset_index(drop=True, inplace=True)
cycle_4.reset_index(drop=True, inplace=True)

# Add a column that accumulates the daily percentage change for each cycle
cycle_1['cumulative_pct_change'] = cycle_1['daily_pct_change'].cumsum()
cycle_2['cumulative_pct_change'] = cycle_2['daily_pct_change'].cumsum()
cycle_3['cumulative_pct_change'] = cycle_3['daily_pct_change'].cumsum()
cycle_4['cumulative_pct_change'] = cycle_4['daily_pct_change'].cumsum()
cycle_2

C:\Users\chris\AppData\Local\Temp\ipykernel_5088\2507391877.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\chris\AppData\Local\Temp\ipykernel_5088\2507391877.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\chris\AppData\Local\Temp\ipykernel_5088\2507391877.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,date,price,daily_pct_change,cumulative_pct_change
0,2015-01-14,164.9,-25.485766,-25.485766
1,2015-01-15,209.8,27.228623,1.742857
2,2015-01-16,206.9,-1.382269,0.360589
3,2015-01-17,199.6,-3.528275,-3.167686
4,2015-01-18,210.6,5.511022,2.343336
...,...,...,...,...
1064,2018-01-02,14754.1,9.737521,496.451666
1065,2018-01-03,15156.6,2.728055,499.179722
1066,2018-01-04,15180.1,0.155048,499.334770
1067,2018-01-05,16954.8,11.690964,511.025733
